In [ ]:
# Data manipulation
#===========================================================================
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np

# Data visualization
#===========================================================================
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

#
#===========================================================================
import random
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
# Get the name of the first uploaded file
filename = next(iter(uploaded))

In [ ]:
data = pd.read_csv(filename)

In [ ]:
data.head()

In [ ]:
data['Full Name'] = data['LastName'] + ', ' + data['PreferredName']

In [ ]:
# Set a seed for reproducibility
random.seed(42)  # You can choose any integer as a seed

# List of random words
animals = ['Lion', 'Tiger', 'Bear', 'Eagle', 'Shark', 'Dragon', 'Jaguar', 'Falcon', 'Dolphin']
colors = ['Crimson', 'Cyan', 'Aqua', 'Aureolin', 'Cerulean', 'Chartreuse', 'Vermilion']

# Create coded names
def generate_coded_name(student_name):
    animal = random.choice(animals)
    color = random.choice(colors)
    unique_number = random.randint(100, 999)  # Unique number
    return f"{color}{animal}{unique_number}"


# Apply the function to create a new column for coded names
data['Coded Name'] = data['Full Name'].apply(generate_coded_name)



In [ ]:
# Identify all columns containing '%'
percent_columns = [col for col in data.columns if '%' in col]

# Process each percentage column
for col in percent_columns:
    # Create TDifference
    target_diff_col = col.replace('%', 'TDifference')
    data[target_diff_col] = data[col] - data['Prediction']

    # Create RTDifference
    relative_diff_col = col.replace('%', 'RTDifference')
    data[relative_diff_col] = data[target_diff_col] - data[target_diff_col].mean()

In [ ]:
data['TargetDelta'] = data.iloc[:, -1] - data.iloc[:, -3]

In [ ]:
data['latest_rt_difference'] = data.iloc[:, -2]  # Second to last column

In [ ]:
# Get the name of the second-to-last column
third_to_last_column_name = data.columns[-3]  # Access third to last column name
first_four_letters = third_to_last_column_name[:4]  # Get first four letters

# Get the name of the second-to-last column
fifth_to_last_column_name = data.columns[-5]  # Access fifth to last column
first_four_letters2 = fifth_to_last_column_name[:4]  # Get first four letters

In [ ]:
# Calculate distance from origin
data['Distance'] = np.sqrt(data['latest_rt_difference']**2 + data['TargetDelta']**2)

# Normalize the distances
scaler = MinMaxScaler()
data['Normalized Distance'] = scaler.fit_transform(data[['Distance']])

# Classify points by quadrant
def get_quadrant(row):
    if row['latest_rt_difference'] >= 0 and row['TargetDelta'] >= 0:
        return 'Quadrant 1'  # Top right
    elif row['latest_rt_difference'] < 0 and row['TargetDelta'] >= 0:
        return 'Quadrant 2'  # Top left
    elif row['latest_rt_difference'] < 0 and row['TargetDelta'] < 0:
        return 'Quadrant 3'  # Bottom left
    else:
        return 'Quadrant 4'  # Bottom right

data['Quadrant'] = data.apply(get_quadrant, axis=1)

# Define custom colormaps for each quadrant
q1_colors = ['#8ac7ff', '#000f96']  # Light blue to dark blue
q2_colors = ['#92fc92', '#008c00']  # Light green to dark green
q3_colors = ['#fab6b6', '#e60202']  # Light red to dark red
q4_colors = ['#fced86', '#fffb03']  # Light orange to dark orange

q1_cmap = LinearSegmentedColormap.from_list("Q1", q1_colors)
q2_cmap = LinearSegmentedColormap.from_list("Q2", q2_colors)
q3_cmap = LinearSegmentedColormap.from_list("Q3", q3_colors)
q4_cmap = LinearSegmentedColormap.from_list("Q4", q4_colors)

# Define colors for each quadrant
def get_color(quadrant, normalized_distance):
    if quadrant == 'Quadrant 1':
        return q1_cmap(normalized_distance)
    elif quadrant == 'Quadrant 2':
        return q2_cmap(normalized_distance)
    elif quadrant == 'Quadrant 3':
        return q3_cmap(normalized_distance)
    else:
        return q4_cmap(normalized_distance)

# Create an array of colors based on quadrant and normalized distance
colors = [get_color(row['Quadrant'], row['Normalized Distance']) for index, row in data.iterrows()]

# Create the scatter plot
plt.figure(figsize=(12, 8))
scatter = plt.scatter(data['latest_rt_difference'], data['TargetDelta'],
                      c=colors, s=100, alpha=0.7)

# Create the title dynamically
title = f'Relative Target Gap vs. Target Delta\nClass Performance Comparing {first_four_letters.upper()} and {first_four_letters2.upper()} Results'

# Add title and labels
plt.title(title, fontsize=16)
plt.xlabel('Relative Target Gap', fontsize=12)
plt.ylabel('Target Delta', fontsize=12)
plt.axhline(0, color='gray', linestyle='--', alpha=0.5)  # Horizontal line at y=0
plt.axvline(0, color='gray', linestyle='--', alpha=0.5)  # Vertical line at x=0

# Add labels to each point
for i, row in data.iterrows():
    plt.annotate(row['Full Name'], (row['latest_rt_difference'], row['TargetDelta']),
                 xytext=(5, 5), textcoords='offset points', fontsize=8,
                 bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.7))

# Add grid lines
plt.grid(color='gray', linestyle='--', alpha=0.3)

# Create a custom legend
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label='Excelling',
               markerfacecolor=q1_colors[1], markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Improving',
               markerfacecolor=q2_colors[1], markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Concerning',
               markerfacecolor=q3_colors[1], markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Declining',
               markerfacecolor=q4_colors[1], markersize=10)
]

plt.legend(handles=legend_elements, loc='lower left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.show()

In [ ]:
# Calculate distance from origin
data['Distance'] = np.sqrt(data['latest_rt_difference']**2 + data['TargetDelta']**2)

# Normalize the distances
scaler = MinMaxScaler()
data['Normalized Distance'] = scaler.fit_transform(data[['Distance']])

# Classify points by quadrant
def get_quadrant(row):
    if row['latest_rt_difference'] >= 0 and row['TargetDelta'] >= 0:
        return 'Quadrant 1'  # Top right
    elif row['latest_rt_difference'] < 0 and row['TargetDelta'] >= 0:
        return 'Quadrant 2'  # Top left
    elif row['latest_rt_difference'] < 0 and row['TargetDelta'] < 0:
        return 'Quadrant 3'  # Bottom left
    else:
        return 'Quadrant 4'  # Bottom right

data['Quadrant'] = data.apply(get_quadrant, axis=1)

# Define custom colormaps for each quadrant
q1_colors = ['#8ac7ff', '#000f96']  # Light blue to dark blue
q2_colors = ['#92fc92', '#008c00']  # Light green to dark green
q3_colors = ['#fab6b6', '#e60202']  # Light red to dark red
q4_colors = ['#fced86', '#fffb03']  # Light orange to dark orange

q1_cmap = LinearSegmentedColormap.from_list("Q1", q1_colors)
q2_cmap = LinearSegmentedColormap.from_list("Q2", q2_colors)
q3_cmap = LinearSegmentedColormap.from_list("Q3", q3_colors)
q4_cmap = LinearSegmentedColormap.from_list("Q4", q4_colors)

# Define colors for each quadrant
def get_color(quadrant, normalized_distance):
    if quadrant == 'Quadrant 1':
        return q1_cmap(normalized_distance)
    elif quadrant == 'Quadrant 2':
        return q2_cmap(normalized_distance)
    elif quadrant == 'Quadrant 3':
        return q3_cmap(normalized_distance)
    else:
        return q4_cmap(normalized_distance)

# Create an array of colors based on quadrant and normalized distance
colors = [get_color(row['Quadrant'], row['Normalized Distance']) for index, row in data.iterrows()]

# Create the scatter plot
plt.figure(figsize=(12, 8))
scatter = plt.scatter(data['latest_rt_difference'], data['TargetDelta'],
                      c=colors, s=100, alpha=0.7)

# Create the title dynamically
title = f'Relative Target Gap vs. Target Delta\nClass Performance Comparing {first_four_letters.upper()} and {first_four_letters2.upper()} Results'

# Add title and labels
plt.title(title, fontsize=16)
plt.xlabel('Relative Target Gap', fontsize=12)
plt.ylabel('Target Delta', fontsize=12)
plt.axhline(0, color='gray', linestyle='--', alpha=0.5)  # Horizontal line at y=0
plt.axvline(0, color='gray', linestyle='--', alpha=0.5)  # Vertical line at x=0

# Add labels to each point
for i, row in data.iterrows():
    plt.annotate(row['Coded Name'], (row['latest_rt_difference'], row['TargetDelta']),
                 xytext=(5, 5), textcoords='offset points', fontsize=8,
                 bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.7))

# Add grid lines
plt.grid(color='gray', linestyle='--', alpha=0.3)

# Create a custom legend
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label='Excelling',
               markerfacecolor=q1_colors[1], markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Improving',
               markerfacecolor=q2_colors[1], markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Concerning',
               markerfacecolor=q3_colors[1], markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Declining',
               markerfacecolor=q4_colors[1], markersize=10)
]

plt.legend(handles=legend_elements, loc='lower left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.show()

In [ ]:
data.head()

In [ ]:
data.to_csv('8 CON ALG2 Analysis Coded Names', index=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import LinearSegmentedColormap

# Sample DataFrame (replace this with your actual DataFrame

# Create the scatter plot
plt.figure(figsize=(12, 8))
scatter = plt.scatter(data['latest_rt_difference'], data['TargetDelta'],
                      c=colors, s=100, alpha=0.7)

# Create the title dynamically
title = f'Relative Target Gap vs. Target Delta\nClass Performance Comparing {first_four_letters.upper()} and {first_four_letters2.upper()} Results'

# Add title and labels
plt.title(title, fontsize=16)
plt.xlabel('Relative Target Gap', fontsize=12)
plt.ylabel('Target Delta', fontsize=12)
plt.axhline(0, color='gray', linestyle='--', alpha=0.5)  # Horizontal line at y=0
plt.axvline(0, color='gray', linestyle='--', alpha=0.5)  # Vertical line at x=0

# Add labels to each point
for i, row in data.iterrows():
    plt.annotate(row['Coded Name'], (row['latest_rt_difference'], row['TargetDelta']),
                 xytext=(5, 5), textcoords='offset points', fontsize=8,
                 bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.7))

# Add grid lines
plt.grid(color='gray', linestyle='--', alpha=0.3)

# Create a custom legend
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label='Excelling',
               markerfacecolor=q1_colors[1], markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Improving',
               markerfacecolor=q2_colors[1], markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Concerning',
               markerfacecolor=q3_colors[1], markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Declining',
               markerfacecolor=q4_colors[1], markersize=10)
]

plt.legend(handles=legend_elements, loc='lower left', bbox_to_anchor=(1, 0.5))

top_students = data.nlargest(5, 'latest_rt_difference')[['Full Name', 'latest_rt_difference']].round(1)

# Define custom column labels
custom_column_labels = ['Student', f'{first_four_letters.upper()} Relative Target Beat By (%)']

# Create a table
table = plt.table(cellText=top_students.values,
                  colLabels=custom_column_labels,  # Use custom labels here
                  cellLoc='center',
                  loc='bottom',
                  bbox=[0.2, -0.3, 0.6, 0.2])  # Adjust bbox as needed

# Adjust the layout to make room for the table
plt.subplots_adjust(bottom=0.3)

plt.tight_layout()
plt.show()